# Visualization notebook



## Imports and functions

In [3]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
import spacy
from spacy import Language
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# 1,071,477 authors

# Exploring MUD

In [90]:
import ijson
import sqlite3
import random

data = ijson.parse(open("data/mud/full/raw_mud/raw_all/data.jsonl"),multiple_values=True)

x = 0
d = {}
posts = []
for prefix, event, value in data:
    
    if prefix == "syms.item":
        posts.append(value)
        
    if prefix.startswith("author_id"):
        d[value] = posts
        posts = []
        x += 1

    if x == 30:
        break



author = random.choice(list(d.keys()))


save = {author:d[author]}
print(len(d[author]))

utils.save_json(save, "example_author.json")

In [1]:
nlp = utils.load_spacy("en_core_web_md")

text = "This is a string and I eat ice cream!!!"
doc = nlp(text)

tokens = [token.text for token in doc]
pos = [token.pos_ for token in doc]

l = []
for i, token in enumerate(tokens):
    try:
        l.append((token, pos[i+1]))
    except:
        pass
    
    
l

NameError: name 'utils' is not defined

In [30]:
nlp = utils.load_spacy("en_core_web_md")

text = "This is a string and I eat ice cream. g"
doc = nlp(text)

arrays = []
for token in doc:
    arrays.append(token.vector)
    
d = np.mean(arrays, axis=0)


list(doc.sents)[0]

This is a string and I eat ice cream.

In [15]:
import nltk

string = "This is is a string".split()

fdist = nltk.FreqDist(string)
fdist.hapaxes()

['This', 'a', 'string']